In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from PIL import Image
from keras.layers import Dense, Flatten, Dropout

import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling3D, Flatten, Dense, Dropout, Input
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [46]:
# Load the dataset into a DataFrame
dataset_path = '/kaggle/input/vodhdataset/sampled_dataset_with_images_vodh (1).csv'

df = pd.read_csv(dataset_path)


In [47]:
df = df.dropna()

In [48]:
df['image_matrix'][2]

'[[[255 253 255]\n  [255 254 255]\n  [254 255 251]\n  ...\n  [253 255 251]\n  [255 255 254]\n  [255 255 255]]\n\n [[255 253 255]\n  [252 249 251]\n  [254 255 251]\n  ...\n  [251 255 249]\n  [255 255 252]\n  [254 255 253]]\n\n [[254 253 255]\n  [255 255 255]\n  [255 255 254]\n  ...\n  [255 255 251]\n  [250 254 248]\n  [253 255 251]]\n\n ...\n\n [[253 254 255]\n  [253 255 255]\n  [255 255 255]\n  ...\n  [255 255 252]\n  [255 255 252]\n  [255 255 251]]\n\n [[251 254 255]\n  [253 254 255]\n  [252 254 255]\n  ...\n  [253 252 254]\n  [255 251 255]\n  [255 253 255]]\n\n [[251 254 255]\n  [251 254 255]\n  [253 254 255]\n  ...\n  [255 253 255]\n  [255 252 255]\n  [255 250 255]]]'

In [49]:
type(df['image_matrix'][2])

str

In [50]:
# Define a function to convert the string representation of image matrices to actual NumPy arrays
def convert_to_array(string_matrix):
    # Remove newline characters and square brackets
    string_matrix = string_matrix.replace('\n', '').replace('[', '').replace(']', '')
    # Split the string by whitespace and convert to integers
    pixel_values = [int(value) for value in string_matrix.split() if value.isdigit()]
    # Convert to NumPy array
    return np.array(pixel_values)


In [51]:
df['image_matrix'] = df['image_matrix'].apply(convert_to_array)

In [52]:
df['image_matrix']

0       [224, 224, 224, 224, 224, 224, 223, 223, 223, ...
1       [171, 188, 209, 170, 187, 208, 168, 187, 208, ...
2       [255, 253, 255, 255, 254, 255, 254, 255, 251, ...
3       [0, 1, 11, 0, 1, 11, 0, 1, 11, 0, 1, 11, 0, 1,...
4       [226, 167, 98, 226, 167, 98, 224, 166, 100, 20...
                              ...                        
2609    [1, 1, 2, 2, 2, 3, 5, 5, 4, 2, 2, 3, 2, 3, 3, ...
2610    [120, 54, 13, 99, 34, 0, 96, 33, 0, 26, 18, 18...
2611    [11, 31, 78, 11, 31, 78, 11, 31, 78, 34, 39, 7...
2612    [228, 225, 227, 229, 226, 228, 229, 226, 228, ...
2613    [34, 19, 16, 34, 19, 16, 34, 19, 16, 34, 20, 2...
Name: image_matrix, Length: 2614, dtype: object

In [53]:
# Check the sizes of the image arrays
sizes = df['image_matrix'].apply(lambda x: x.size)
unique_sizes = sizes.unique()
print("Unique sizes of image arrays:", unique_sizes)

Unique sizes of image arrays: [108]


In [54]:
# Define a function to resize the image arrays
def resize_image_array(image_array):
    # Reshape the image array to a 3D array (height, width, channels)
    image_array = np.array(image_array).reshape((int(len(image_array) / 3), 3))
    
    # Resize the image array to (224, 224) using PIL
    image = Image.fromarray(image_array.astype('uint8'))
    resized_image = image.resize((224, 224))
    
    # Convert the resized image to a NumPy array
    resized_image_array = np.array(resized_image)
    
    # Add a third dimension representing the channels
    resized_image_array = np.expand_dims(resized_image_array, axis=2)
    
    # Stack the array along the third dimension to create (224, 224, 3)
    resized_image_array = np.tile(resized_image_array, (1, 1, 3))
    
    return resized_image_array

In [55]:
df['resized_image_array'] = df['image_matrix'].apply(resize_image_array)

In [56]:
df['resized_image_array'].head()

0    [[[224, 224, 224], [224, 224, 224], [224, 224,...
1    [[[169, 169, 169], [169, 169, 169], [169, 169,...
2    [[[255, 255, 255], [255, 255, 255], [255, 255,...
3    [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...
4    [[[233, 233, 233], [233, 233, 233], [233, 233,...
Name: resized_image_array, dtype: object

In [57]:
print("Shape of image data:", df['resized_image_array'][3].shape)

Shape of image data: (224, 224, 3)


In [58]:
# Define a mapping for the labels
label_mapping = {"TRUE": 1, "Fake": 0}

# Convert string labels to integer labels
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(df['Label'].map(label_mapping))
df['Label'] = integer_labels

In [59]:
# Split the dataset into features (X) and labels (y)
X = np.array(df['resized_image_array'].tolist())  # Features  
y = (df['Label'])

In [60]:
print("Shape of image data:", X[3].shape)

Shape of image data: (224, 224, 3)


In [61]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
print("Shape of image data:", X_train[3].shape)

Shape of image data: (224, 224, 3)


In [63]:
# Load pre-trained VGG16 model without top (fully connected layers)
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
vgg_model.trainable = False

In [64]:
# Create a new model
model = Sequential([
    vgg_model,  # Use pre-trained VGG16 as convolutional base
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Use sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [65]:
# Train the model using the fit method with validation data
model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/4
 1/66 ━━━━━━━━━━━━━━━━━━━━ 2:33 2s/step - accuracy: 0.2188 - loss: 4.6808

W0000 00:00:1711379432.349472     383 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.7632 - loss: 9.4644

W0000 00:00:1711379440.654187     383 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1711379441.572348     386 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 174ms/step - accuracy: 0.7638 - loss: 9.3951 - val_accuracy: 0.8451 - val_loss: 1.2103
Epoch 2/4
 1/66 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - accuracy: 0.8125 - loss: 2.2299

W0000 00:00:1711379443.769867     384 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 149ms/step - accuracy: 0.8749 - loss: 0.6831 - val_accuracy: 0.8394 - val_loss: 1.0085
Epoch 3/4
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 152ms/step - accuracy: 0.8825 - loss: 0.4395 - val_accuracy: 0.7935 - val_loss: 0.8900
Epoch 4/4
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 151ms/step - accuracy: 0.9152 - loss: 0.2932 - val_accuracy: 0.8356 - val_loss: 0.9614


In [66]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test accuracy: {accuracy}')

17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.8348 - loss: 0.9600
Test accuracy: 0.8355640769004822
